<a href="https://colab.research.google.com/github/soohyunme/exercise/blob/main/Code/02_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
cd /content/drive/MyDrive/4조/01.체육공모전/Code

/content/drive/.shortcut-targets-by-id/159Rw9eptrdiP8wihxnfPLcnRcwjx1x8n/4조/01.체육공모전/Code


# 모듈 로드

In [2]:
import pandas as pd
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer


# 데이터 로드

In [3]:
df_origin = pd.read_csv('../Data/04.processed_data.csv',encoding='cp949')
group = pd.read_csv('../Data/user_group.csv',encoding='cp949')
ex_db = pd.read_csv('../Data/exercise_DB.csv',encoding='cp949')

In [4]:
user_db = df_origin[:-20]
sample_user = df_origin[-20:].iloc[:,:-3]

# 함수 정의

## 데이터프레임 변환 함수

In [5]:
# 나이를 연령대로 변환
def age_group(df):
  df['AGRDE_SE'] = df['TEST_AGE'].apply(lambda x : '10대' if x<20 else (\
                                                    '20대' if x<30 else (\
                                                      '30대' if x<40 else (\
                                                    '40대' if x<50 else (\
                                                    '50대' if x<60 else (\
                                                      '60대' if x<70 else (\
                                                    '70대 이상' )))))))
  return df

In [6]:
# 키와 몸무게를 통해 BMI 계산, BMI를 통해 비만 단계 변환
def bmi_group(df):
  bmi = pd.Series(round(df['WEIGHT'] / (df['HEIGHT']/100)**2,1),name='BMI')
  bmi_g = bmi.apply(lambda x : '3단계비만' if x>=35 else (\
                                                    '2단계비만' if x>=30 else (\
                                                      '1단계비만' if x>=25 else (\
                                                    '비만전단계' if x>=23 else (\
                                                    '정상' if x>=18.5 else '저체중')))))
  result = pd.concat([df,bmi_g],axis=1)
  return result

In [7]:
# Add user group column
def add_group(df):
  result = df.merge(group,how='inner')
  return result

In [8]:
def transform(df):
  result = age_group(df.copy())
  result = bmi_group(result)
  result = add_group(result)
  if '준비운동' in df.columns:
    result = result[['USER_ID', 'GROUP_ID', '준비운동', '본운동', '마무리운동']]
  else:
    result = result[['USER_ID', 'GROUP_ID']]
  return result

## 최빈 운동 세트 선정 함수

In [9]:
# select top 1 exercise set from user info
# input : group_id / ouput : top 1 exercise set(pre, main, finish / Series)
def mode_set(group_id):
  user_set = set_df[set_df['GROUP_ID']==g_id]
  result = user_set[user_set['COUNT'] == user_set['COUNT'].max()].iloc[0,1:4]
  return result


## 코사인 유사도 함수

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer() # Counter Vectorizer 객체 생성 
# 운동을 Counter Vectorizer 형태로 변형 
#countvect = vect.fit_transform(None)
#countvect 
#vect.vocabulary_

In [11]:
# 운동을 Counter Vectorizer 형태로 변형 
#countvect = vect.fit_transform(aa)
#countvect 
#vect.vocabulary_

# 데이터셋 변환  
연령대, 운동강도, 성별, BMI 등급을 통한 그룹 변환

In [12]:
# User DB & sample user data transform
trans_db = transform(user_db)
trans_sample = transform(sample_user)

# exercise set df
set_df = pd.DataFrame(trans_db.drop('USER_ID',axis=1).value_counts()).reset_index().rename(columns={0:'COUNT'})

# 맞춤 운동 추천

In [13]:
for u_id, g_id in trans_sample.values:
  print(list(mode_set(g_id))) # 준비, 본, 마무리

['걷기', '달리기,무릎 높여 제자리 달리기', '전신 루틴 스트레칭']
['걷기', '달리기,무릎 높여 제자리 달리기', '전신 루틴 스트레칭']
['걷기', '달리기,무릎 높여 제자리 달리기', '전신 루틴 스트레칭']
['걷기', '달리기,무릎 높여 제자리 달리기', '전신 루틴 스트레칭']
['걷기', '달리기,무릎 높여 제자리 달리기', '전신 루틴 스트레칭']
['달리기,동적 스트레칭 루틴프로그램,팔 벌려 뛰기', '맨몸운동  루틴프로그램,버피운동,메디신 볼 잡고 팔굽혀펴기,종아리 사이 감아 돌리기,볼 잡고 점프하기,순간반응 콘 찍기,뒤꿈치 높이들어 뛰기,사다리 이용 루틴 운동,다리 뻗어 점프하기', '걷기,정적 스트레칭  루틴프로그램']
['달리기,동적 스트레칭 루틴프로그램,팔 벌려 뛰기', '맨몸운동  루틴프로그램,버피운동,메디신 볼 잡고 팔굽혀펴기,종아리 사이 감아 돌리기,볼 잡고 점프하기,순간반응 콘 찍기,뒤꿈치 높이들어 뛰기,사다리 이용 루틴 운동,다리 뻗어 점프하기', '걷기,정적 스트레칭  루틴프로그램']
['걷기', '달리기,무릎 높여 제자리 달리기', '전신 루틴 스트레칭']
['팔꿈치 원 그리기,정적 스트레칭  루틴프로그램,깍지끼고 목 들어올리기,팔꿈치 머리 뒤로 젖히기,양발 벌려 무릎 밀어내기,온 몸 뻗어주기', '맨몸운동  루틴프로그램,저항밴드 운동  루틴프로그램,전완대고 버티기,앉았다 일어서기,발 바꿔 뛰기,버피운동,걷기,달리기,자전거타기,줄넘기,수영,사이드 지그재그 뛰기,순간반응 콘 찍기,뒤꿈치 높이들어 뛰기,무릎 높이 들어 뛰기,팔 벌려 뛰기,앉았다 일어나며 점프하기,요가 및 필라테스  루틴프로그램,측면 팔대고 버티기,옆으로 팔 벌리며 점프하기,팔 돌리며 점프하기,다리 들어올리기,다리자세', '정적 스트레칭  루틴프로그램,아기자세,온 몸 뻗어주기,휴식자세']
['달리기,동적 스트레칭 루틴프로그램,팔 벌려 뛰기', '맨몸운동  루틴프로그램,버피운동,메디신 볼 잡고 팔굽혀펴기,종아리 사이 감아 돌리기,볼 잡고 점프하기,순간반

In [14]:
mode_set(g_id)

준비운동           걷기
본운동      엉덩이 스트레칭
마무리운동       자전거타기
Name: 55, dtype: object

토크나이저 하는중

g_id 받아서 그거로 토크나이저 진행  
코사인 유사도 구해서 추천 운동 리턴

In [37]:
length = 10
ex_list = list(user_db['준비운동'].apply(lambda x : x.replace(' ','')).apply(lambda x : x.split(',')))[:length]


In [38]:
# 운동 토크나이징
tokenizer = Tokenizer()
preprocessed_sentences = ex_list
tokenizer.fit_on_texts(preprocessed_sentences)
ex_count = tokenizer.word_counts
ex_encoding = tokenizer.texts_to_sequences(preprocessed_sentences)


In [39]:
# 최대 빈도 운동
result_ex_nm = ''
max = -1
for ex_name in ex_count:
  if ex_count[ex_name] >= max :
    max = ex_count[ex_name]
    result_ex_nm = ex_name
print(result_ex_nm)

걷기


In [40]:
user_db['준비운동'].value_counts()

걷기                                                           19900
유산소 운동 전 동적 루틴 스트레칭                                          15671
하지 루틴 스트레칭,상지 루틴 스트레칭                                        13151
전신 루틴 스트레칭                                                   12910
동적 스트레칭 루틴프로그램                                                4868
                                                             ...  
하지 루틴 스트레칭,상지 루틴 스트레칭,유산소 운동 전 동적 루틴 스트레칭,자가근막이완술 루틴 스트레칭        1
유산소 운동 전 동적 루틴 스트레칭,팔벌려뛰기,버피운동,앉았다 일어서기                          1
전신 루틴 스트레칭,유산소 운동 전 동적 루틴 스트레칭,상지 루틴 스트레칭,하지 루틴 스트레칭             1
하지 루틴 스트레칭,상지 루틴 스트레칭,전신 루틴 스트레칭,자가근막이완술 루틴 스트레칭                 1
누워서 전신 뻗기,아래 팔 스트레칭,허리 스트레칭,배 스트레칭,회전근개 스트레칭                     1
Name: 준비운동, Length: 1705, dtype: int64

In [41]:
print(tokenizer.word_index) # 인덱스
print(tokenizer.word_counts) # 카운터
print(ex_encoding)

{'걷기': 1, '유산소운동전동적루틴스트레칭': 2, '자가근막이완술루틴스트레칭': 3, '전신루틴스트레칭': 4, '아래팔스트레칭': 5, '엉덩이스트레칭': 6, '넙다리스트레칭': 7, '허리스트레칭': 8, '배스트레칭': 9, '옆구리스트레칭': 10, '어깨스트레칭': 11}
OrderedDict([('자가근막이완술루틴스트레칭', 1), ('걷기', 5), ('전신루틴스트레칭', 1), ('유산소운동전동적루틴스트레칭', 3), ('아래팔스트레칭', 1), ('엉덩이스트레칭', 1), ('넙다리스트레칭', 1), ('허리스트레칭', 1), ('배스트레칭', 1), ('옆구리스트레칭', 1), ('어깨스트레칭', 1)])
[[3], [1], [4, 2], [1], [1], [2], [1], [1], [2], [5, 6, 7, 8, 9, 10, 11]]


In [42]:
w_list = []
for row  in ex_encoding:
  w_list.append(', '.join(str(_) for _ in row))
w_list


['3', '1', '4, 2', '1', '1', '2', '1', '1', '2', '5, 6, 7, 8, 9, 10, 11']

In [43]:
# 문장을 Counter Vectorizer 형태로 변형 
countvect = vect.fit_transform(w_list[:length])
countvect # 4x9 : 4개의 문서에 9개의 단어 

<10x2 sparse matrix of type '<class 'numpy.int64'>'
	with 2 stored elements in Compressed Sparse Row format>

In [44]:
# toarray()를 통해서 문장이 Vector 형태의 값을 얻을 수 있음 
# 하지만, 각 인덱스와 컬럼이 무엇을 의미하는지에 대해서는 알 수가 없음 
countvect.toarray().shape

(10, 2)

In [45]:
ex_list

[['자가근막이완술루틴스트레칭'],
 ['걷기'],
 ['전신루틴스트레칭', '유산소운동전동적루틴스트레칭'],
 ['걷기'],
 ['걷기'],
 ['유산소운동전동적루틴스트레칭'],
 ['걷기'],
 ['걷기'],
 ['유산소운동전동적루틴스트레칭'],
 ['아래팔스트레칭', '엉덩이스트레칭', '넙다리스트레칭', '허리스트레칭', '배스트레칭', '옆구리스트레칭', '어깨스트레칭']]

In [46]:
tokenizer.word_index

{'걷기': 1,
 '넙다리스트레칭': 7,
 '배스트레칭': 9,
 '아래팔스트레칭': 5,
 '어깨스트레칭': 11,
 '엉덩이스트레칭': 6,
 '옆구리스트레칭': 10,
 '유산소운동전동적루틴스트레칭': 2,
 '자가근막이완술루틴스트레칭': 3,
 '전신루틴스트레칭': 4,
 '허리스트레칭': 8}

In [47]:
vect.vocabulary_

{'10': 0, '11': 1}

In [48]:
countvect.toarray()

array([[0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [1, 1]])

In [66]:
import pandas as pd
countvect_df = pd.DataFrame(countvect.toarray(), columns = sorted(vect.vocabulary_))
countvect_df.index = w_list[:length]
pd.concat([countvect_df.reset_index(),pd.DataFrame(aa_list)])

,0,10,11,index
0,NaN,0.0,0.0,3
1,NaN,0.0,0.0,1
2,NaN,0.0,0.0,"4, 2"
3,NaN,0.0,0.0,1
4,NaN,0.0,0.0,1
5,NaN,0.0,0.0,2
6,NaN,0.0,0.0,1
7,NaN,0.0,0.0,1
8,NaN,0.0,0.0,2
9,NaN,1.0,1.0,"5, 6, 7, 8, 9, 10, 11"


In [71]:
pd.concat([countvect_df.reset_index(),pd.DataFrame(aa_list)],axis=1).to_csv('sadf.csv',encoding='cp949')

In [68]:
  pd.DataFrame(aa_list)

,0
0,자가근막이완술루틴스트레칭
1,걷기
2,"전신루틴스트레칭, 유산소운동전동적루틴스트레칭"
3,걷기
4,걷기
5,유산소운동전동적루틴스트레칭
6,걷기
7,걷기
8,유산소운동전동적루틴스트레칭
9,"아래팔스트레칭, 엉덩이스트레칭, 넙다리스트레칭, 허리스트레칭, 배스트레칭, 옆구리스..."


In [58]:
aa_list = []
for row  in ex_list[:10]:
  aa_list.append(', '.join(str(_) for _ in row))
aa_list

['자가근막이완술루틴스트레칭',
 '걷기',
 '전신루틴스트레칭, 유산소운동전동적루틴스트레칭',
 '걷기',
 '걷기',
 '유산소운동전동적루틴스트레칭',
 '걷기',
 '걷기',
 '유산소운동전동적루틴스트레칭',
 '아래팔스트레칭, 엉덩이스트레칭, 넙다리스트레칭, 허리스트레칭, 배스트레칭, 옆구리스트레칭, 어깨스트레칭']

In [141]:

countvect_df = pd.DataFrame(countvect.toarray(), columns = sorted(vect.vocabulary_))
countvect_df.index = ['문서1', '문서2', '문서3', '문서4']
countvect_df

In [ ]:
vect.vocabulary_ = tokenizer.word_counts

In [ ]:
vect.vocabulary_

In [ ]:
# 운동을 Counter Vectorizer 형태로 변형 
countvect = vect.fit_transform(aa)
countvect 
vect.vocabulary_